## Using Boosting for Estimating Nuisance Parameters in GRF 

## Summary

Using the DGP from the GRF Github ReadMe, I evaluated three methods of predicting E(Y|X) and E(W|X) for input in grf's causal forest: 

- Using a single random forest (existing implementation) 
- Using boosted regression trees (common boosting format)
- Using boosted random forests with learning rate of 1 (as in Stefan's email) 

I evaluate each method using the MSE of Y.hat and E(Y|X), of W.hat and E(W|X), and of Tau.hat and TAU from the resulting causal forest, for each of the three methodologies. 

In [2]:
mse.y = c(mse.y.rf, mse.y.boost,mse.y.both)
mse.w = c(mse.w.rf,mse.w.boost,mse.w.both)
mse.tau = c(mse.tau.rf,mse.tau.boost,mse.tau.both)
data.frame(method=c("Random Forest","Boosted Trees", "Boosted Forests"),mse.y= mse.y,mse.w=mse.w,mse.tau=mse.tau)

ERROR: Error in eval(expr, envir, enclos): object 'mse.y.rf' not found


Boosted forests improve estimation of E(Y|X) significantly. The estimation of E(W|X) improves vs random forests but not vs boosted trees. The MSE of Tau actually increases from random forests to boosted forests. 

The rest of the notebook contains the code and further details on the results. I also print the results for test_calibration for each of the three methods. 

## Details

For now I have relied on as simple R implementation of boosted forests with shrinkage of 1 to estimate Y.hat and W.hat. If we were to add this functionality to GRF, I likely would write a C++ implementation using the existing regression_tree function in GRF and user-specified boosting-specific parameters (like learning rate and the number of forests used). 

In [1]:
library(gbm)
library(grf)
set.seed(10)

Loaded gbm 2.1.4


#### 1. Baseline using regression_forest
Modified this DGP from GRF Github Readme.  

In [2]:
n = 500; p = 10
X = matrix(rnorm(n * p), n, p)
TAU = 1 / (1 + exp(-X[, 3]))
E.W = 1 / (1 + exp(-X[, 1] - X[, 2]))
W = rbinom(n ,1, E.W)
E.Y = pmax(X[, 2] + X[, 3], 0) + rowMeans(X[, 4:6]) / 2 + W * TAU
Y = E.Y + rnorm(n)

In [4]:
forest.W = regression_forest(X, W, tune.parameters = TRUE)
W.hat = predict(forest.W)$predictions

forest.Y = regression_forest(X, Y, tune.parameters = TRUE)
Y.hat = predict(forest.Y)$predictions

str(forest.W)
forest.W$tunable.params

List of 10
 $ serialized.forest: raw [1:13156560] d0 07 00 00 ...
 $ num.trees        : num 2000
 $ min.node.size    : num 1
 $ ci.group.size    : num 2
 $ X.orig           : num [1:500, 1:10] 0.0187 -0.1843 -1.3713 -0.5992 0.2945 ...
 $ Y.orig           : int [1:500] 0 0 0 0 1 1 0 1 0 0 ...
 $ clusters         : num(0) 
 $ tunable.params   : Named num [1:5] 0.5 1 6 0.152 2.266
  ..- attr(*, "names")= chr [1:5] "sample.fraction" "min.node.size" "mtry" "alpha" ...
 $ predictions      : num [1:500] 0.535 0.383 0.302 0.38 0.624 ...
 $ debiased.error   : num [1:500] 0.2857 0.1465 0.0913 0.1443 0.1409 ...
 - attr(*, "class")= chr [1:2] "regression_forest" "grf"


sample.fraction     min.node.size              mtry             alpha 
        0.5000000         1.0000000         6.0000000         0.1518183 
imbalance.penalty 
        2.2662142

[[1]]
[1] "hi"

[[2]]
NULL

[[3]]
[1] "bye"

In [101]:
tau.forest = causal_forest(X, Y, W,
                           W.hat = W.hat, Y.hat = Y.hat,
                           tune.parameters = TRUE)

In [102]:
test_calibration(tau.forest)


Best linear fit using forest predictions (on held-out data)
as well as the mean forest prediction as regressors, along
with heteroskedasticity-robust (HC3) SEs:

                               Estimate Std. Error t value  Pr(>|t|)    
mean.forest.prediction          1.01628    0.16459  6.1746 1.377e-09 ***
differential.forest.prediction  0.49438    0.78946  0.6262    0.5315    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [103]:
tau.hat.forest = predict(tau.forest)$predictions
mse.tau.rf = mean((TAU - tau.hat.forest)^2)

#### 2. Boosted trees don't work as well as the random forest regression alone 

In [104]:
dev.off() 
boost.Y = gbm(formula = Y~ .,
              distribution = "gaussian",
              data = data.frame(Y=Y,X=X),
                 n.trees = 2000,
                  shrinkage = 0.1, 
                  cv.folds = 5,
                  n.cores = 1)

Y.hat.boost = predict(object=boost.Y,n.trees=gbm.perf(boost.Y),type="response")

boost.W = gbm(formula = W~ .,
              distribution = "bernoulli",
              data = data.frame(W=W,X=X),
                 n.trees = 2000,
                  shrinkage = 0.1, 
                  cv.folds = 5,
                  n.cores = 1)

W.hat.boost = predict(object=boost.W,n.trees=gbm.perf(boost.W),type="response")

null device 
          1

In [105]:
boost.tau.forest = causal_forest(X,Y,W,
                                W.hat=W.hat.boost,Y.hat=Y.hat.boost,
                                tune.parameters = TRUE) 

In [106]:
mse.y.rf = mean((Y.hat-E.Y)^2)
mse.y.boost = mean((Y.hat.boost-E.Y)^2)

mse.w.rf = mean((W.hat-E.W)^2)
mse.w.boost = mean((W.hat.boost-E.W)^2)

In [107]:
test_calibration(boost.tau.forest)


Best linear fit using forest predictions (on held-out data)
as well as the mean forest prediction as regressors, along
with heteroskedasticity-robust (HC3) SEs:

                                Estimate Std. Error t value  Pr(>|t|)    
mean.forest.prediction           0.91374    0.17411  5.2480  2.28e-07 ***
differential.forest.prediction -57.48519    6.51384 -8.8251 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [108]:
tau.hat.forest.boost = predict(boost.tau.forest)$predictions
mse.tau.boost = mean((TAU - tau.hat.forest.boost)^2)

I find that the differential.forest.prediction is lower using boosting of regression trees compared to when the random forests are used to predict Y.hat and W.hat. The prediction of E.Y is worse as well while the prediction of E.W. is a bit better for forests. The prediction of tau is substantially worse. 

#### Boosted random forests have mixed results vs random forests alone 

In [109]:
boosted_forest_predictions <- function(Y,X,num_trees) {
    
    forest_1 = regression_forest(X,Y)
    e.hat = forest_1$predictions
    e = Y- e.hat
    Y.hat = e.hat

    for (i in 2:num_trees) {
        forest = regression_forest(X,e)
        e.hat = forest$predictions
        e = e - e.hat 
        Y.hat = Y.hat+ e.hat 
    }
    return(Y.hat)  
}

In [110]:
Y.hat.both = boosted_forest_predictions(Y,X,4)

In [111]:
W.hat.both = boosted_forest_predictions(W,X,4)

In [112]:
mse.y.both = mean((Y.hat.both-E.Y)^2)
mse.w.both = mean((W.hat.both - E.W)^2)

In [113]:
both.tau.forest = causal_forest(X,Y,W,
                                W.hat=W.hat.both,Y.hat=Y.hat.both,
                                tune.parameters = TRUE) 
test_calibration(boost.tau.forest)


Best linear fit using forest predictions (on held-out data)
as well as the mean forest prediction as regressors, along
with heteroskedasticity-robust (HC3) SEs:

                                Estimate Std. Error t value  Pr(>|t|)    
mean.forest.prediction           0.91374    0.17411  5.2480  2.28e-07 ***
differential.forest.prediction -57.48519    6.51384 -8.8251 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [114]:
tau.hat.forest.both = predict(both.tau.forest)$predictions
mse.tau.both = mean((TAU - tau.hat.forest.both)^2)

[1] 0.02866853